In [1]:
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader

def get_data_loaders(batch_size=128):
    transform = transforms.Compose([
        transforms.Resize(32),
        transforms.ToTensor(),
    ])

    train_dataset = datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)
    test_dataset = datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

    return train_loader, test_loader

# 获取数据加载器
train_loader, test_loader = get_data_loaders()


Files already downloaded and verified
Files already downloaded and verified


In [2]:
import torch
import torch.nn as nn

class Generator(nn.Module):
    def __init__(self, z_dim):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(z_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Linear(1024, 3 * 32 * 32),
            nn.Tanh()
        )

    def forward(self, z):
        return self.model(z).view(-1, 3, 32, 32)

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(3 * 32 * 32, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 1),
            nn.Sigmoid()
        )

    def forward(self, img):
        return self.model(img.view(-1, 3 * 32 * 32))

# 创建模型实例
z_dim = 100
generator = Generator(z_dim)
discriminator = Discriminator()


In [3]:
def train_gan(generator, discriminator, train_loader, num_epochs, device):
    criterion = nn.BCELoss()
    g_optimizer = torch.optim.Adam(generator.parameters(), lr=0.0002)
    d_optimizer = torch.optim.Adam(discriminator.parameters(), lr=0.0002)

    generator.train()
    discriminator.train()

    for epoch in range(num_epochs):
        for real_imgs, _ in train_loader:
            real_imgs = real_imgs.to(device)

            # 训练鉴别器
            z = torch.randn(real_imgs.size(0), z_dim).to(device)
            fake_imgs = generator(z)
            real_labels = torch.ones(real_imgs.size(0), 1).to(device)
            fake_labels = torch.zeros(real_imgs.size(0), 1).to(device)

            d_loss_real = criterion(discriminator(real_imgs), real_labels)
            d_loss_fake = criterion(discriminator(fake_imgs.detach()), fake_labels)
            d_loss = d_loss_real + d_loss_fake

            d_optimizer.zero_grad()
            d_loss.backward()
            d_optimizer.step()

            # 训练生成器
            g_loss = criterion(discriminator(fake_imgs), real_labels)

            g_optimizer.zero_grad()
            g_loss.backward()
            g_optimizer.step()

        print(f'Epoch [{epoch+1}/{num_epochs}], d_loss: {d_loss.item():.4f}, g_loss: {g_loss.item():.4f}')

# 确定设备
device = 'cuda' if torch.cuda.is_available() else 'cpu'
generator.to(device)
discriminator.to(device)

# 训练GAN
train_gan(generator, discriminator, train_loader, num_epochs=50, device=device)


Epoch [1/50], d_loss: 4.2560, g_loss: 1.3535
Epoch [2/50], d_loss: 2.1416, g_loss: 7.5597
Epoch [3/50], d_loss: 1.3210, g_loss: 3.9660
Epoch [4/50], d_loss: 0.3766, g_loss: 7.1729
Epoch [5/50], d_loss: 1.3486, g_loss: 2.5812
Epoch [6/50], d_loss: 1.1296, g_loss: 1.3571
Epoch [7/50], d_loss: 0.7268, g_loss: 2.4430
Epoch [8/50], d_loss: 1.4075, g_loss: 2.5808
Epoch [9/50], d_loss: 0.8928, g_loss: 3.2598
Epoch [10/50], d_loss: 0.3123, g_loss: 2.6101
Epoch [11/50], d_loss: 0.7124, g_loss: 2.5552
Epoch [12/50], d_loss: 1.1040, g_loss: 0.9410
Epoch [13/50], d_loss: 0.9925, g_loss: 1.2579
Epoch [14/50], d_loss: 1.6268, g_loss: 1.2360
Epoch [15/50], d_loss: 1.1316, g_loss: 0.8849
Epoch [16/50], d_loss: 1.7553, g_loss: 1.8524
Epoch [17/50], d_loss: 0.9972, g_loss: 0.9320
Epoch [18/50], d_loss: 0.8135, g_loss: 1.1234
Epoch [19/50], d_loss: 0.6311, g_loss: 2.3820
Epoch [20/50], d_loss: 1.2199, g_loss: 1.4780
Epoch [21/50], d_loss: 0.4374, g_loss: 1.9819
Epoch [22/50], d_loss: 0.7181, g_loss: 1.78

In [4]:
import torch
import torchvision.models as models
from torch.utils.tensorboard import SummaryWriter

def train_resnet(model, train_loader, test_loader, num_epochs, device):
    writer = SummaryWriter('runs/cifar100_gan_resnet')
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(num_epochs):
        model.train()
        for imgs, labels in train_loader:
            imgs, labels = imgs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        # 记录损失
        writer.add_scalar('Loss/train', loss.item(), epoch)

        # 测试阶段
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for imgs, labels in test_loader:
                imgs, labels = imgs.to(device), labels.to(device)
                outputs = model(imgs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / total
        writer.add_scalar('Accuracy/test', accuracy, epoch)
        print(f'Epoch [{epoch+1}/{num_epochs}], Accuracy: {accuracy:.2f}%')

    writer.close()

# 初始化ResNet-18
resnet18 = models.resnet18(weights='IMAGENET1K_V1')  # 更新这里
resnet18.fc = nn.Linear(resnet18.fc.in_features, 100)  # 修改输出层
resnet18 = resnet18.to(device)

# 使用生成的数据训练ResNet-18
train_resnet(resnet18, train_loader, test_loader, num_epochs=50, device=device)


Epoch [1/50], Accuracy: 36.29%
Epoch [2/50], Accuracy: 45.90%
Epoch [3/50], Accuracy: 45.68%
Epoch [4/50], Accuracy: 49.22%
Epoch [5/50], Accuracy: 50.95%
Epoch [6/50], Accuracy: 49.30%
Epoch [7/50], Accuracy: 51.27%
Epoch [8/50], Accuracy: 50.97%
Epoch [9/50], Accuracy: 51.01%
Epoch [10/50], Accuracy: 50.68%
Epoch [11/50], Accuracy: 50.59%
Epoch [12/50], Accuracy: 50.67%
Epoch [13/50], Accuracy: 51.78%
Epoch [14/50], Accuracy: 51.10%
Epoch [15/50], Accuracy: 50.71%
Epoch [16/50], Accuracy: 50.96%
Epoch [17/50], Accuracy: 50.37%
Epoch [18/50], Accuracy: 51.45%
Epoch [19/50], Accuracy: 52.64%
Epoch [20/50], Accuracy: 50.31%
Epoch [21/50], Accuracy: 49.06%
Epoch [22/50], Accuracy: 51.16%
Epoch [23/50], Accuracy: 52.63%
Epoch [24/50], Accuracy: 49.15%
Epoch [25/50], Accuracy: 52.05%
Epoch [26/50], Accuracy: 51.13%
Epoch [27/50], Accuracy: 50.72%
Epoch [28/50], Accuracy: 50.97%
Epoch [29/50], Accuracy: 51.14%
Epoch [30/50], Accuracy: 51.07%
Epoch [31/50], Accuracy: 50.16%
Epoch [32/50], Ac

In [5]:
# 初始化一个新的ResNet-18模型（从零开始）
resnet18_zero = models.resnet18(weights=None)  # 从零开始训练
resnet18_zero.fc = nn.Linear(resnet18_zero.fc.in_features, 100)  # 修改输出层
resnet18_zero = resnet18_zero.to(device)

# 使用训练数据训练ResNet-18（从零开始）
train_resnet(resnet18_zero, train_loader, test_loader, num_epochs=50, device=device)


Epoch [1/50], Accuracy: 19.11%
Epoch [2/50], Accuracy: 27.26%
Epoch [3/50], Accuracy: 32.81%
Epoch [4/50], Accuracy: 33.93%
Epoch [5/50], Accuracy: 40.43%
Epoch [6/50], Accuracy: 38.78%
Epoch [7/50], Accuracy: 34.91%
Epoch [8/50], Accuracy: 42.67%
Epoch [9/50], Accuracy: 41.67%
Epoch [10/50], Accuracy: 43.55%
Epoch [11/50], Accuracy: 38.62%
Epoch [12/50], Accuracy: 40.88%
Epoch [13/50], Accuracy: 41.43%
Epoch [14/50], Accuracy: 41.99%
Epoch [15/50], Accuracy: 41.98%
Epoch [16/50], Accuracy: 42.60%
Epoch [17/50], Accuracy: 40.67%
Epoch [18/50], Accuracy: 42.52%
Epoch [19/50], Accuracy: 39.54%
Epoch [20/50], Accuracy: 41.68%
Epoch [21/50], Accuracy: 43.02%
Epoch [22/50], Accuracy: 43.85%
Epoch [23/50], Accuracy: 41.14%
Epoch [24/50], Accuracy: 43.63%
Epoch [25/50], Accuracy: 42.67%
Epoch [26/50], Accuracy: 42.27%
Epoch [27/50], Accuracy: 43.85%
Epoch [28/50], Accuracy: 42.65%
Epoch [29/50], Accuracy: 42.08%
Epoch [30/50], Accuracy: 43.47%
Epoch [31/50], Accuracy: 42.84%
Epoch [32/50], Ac

In [6]:
# 加载ImageNet预训练的ResNet-18
imagenet_resnet18 = models.resnet18(weights='IMAGENET1K_V1')
imagenet_resnet18.fc = nn.Linear(imagenet_resnet18.fc.in_features, 100)  # 修改输出层
imagenet_resnet18 = imagenet_resnet18.to(device)


In [9]:
def evaluate_model(model, test_loader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for imgs, labels in test_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

# 评估三个模型的准确率
gan_resnet_accuracy = evaluate_model(resnet18, test_loader, device)  # GAN生成数据
zero_resnet_accuracy = evaluate_model(resnet18_zero, test_loader, device)  # 从零开始训练

print(f'GAN ResNet Accuracy: {gan_resnet_accuracy:.2f}%')
print(f'Zero ResNet Accuracy: {zero_resnet_accuracy:.2f}%')


GAN ResNet Accuracy: 51.66%
Zero ResNet Accuracy: 43.23%


In [10]:
# 汇总和比较结果
print("\n模型比较结果:")
print(f"GAN生成的ResNet-18准确率: {gan_resnet_accuracy:.2f}%")
print(f"从零开始训练的ResNet-18准确率: {zero_resnet_accuracy:.2f}%")



模型比较结果:
GAN生成的ResNet-18准确率: 51.66%
从零开始训练的ResNet-18准确率: 43.23%
